In [ ]:
!pip install keras==2.2.4
!pip install tensorflow

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='4zdc_CjgpF4rUb1Hy66H1Pw8DtZoOw_nDg-3W1JB8rUt',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'smartlender-donotdelete-pr-luukgx3bidqosk'
object_key = 'test.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )



test = pd.read_csv(body)
test.head()

In [ ]:


bucket = 'smartlender-donotdelete-pr-luukgx3bidqosk'
object_key = 'train.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train= pd.read_csv(body)
train.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import pickle

In [ ]:
scaler = MaxAbsScaler()

In [ ]:
train.head()

In [ ]:
train_y = train.iloc[:,-1]

In [ ]:
train_x = train.drop('Loan_Status',axis=1)

In [ ]:
test_y = test.iloc[:,-1]

In [ ]:
test_x = test.drop('Loan_Status',axis=1)

In [ ]:
x = pd.concat([train_x,test_x],axis=0)
y = pd.concat([train_y,test_y],axis=0)

In [ ]:
train_x = scaler.fit_transform(train_x)

In [ ]:
test_x = scaler.transform(test_x)

In [ ]:
def decisionTree(train_x,test_x,train_y,test_y):
    dt = DecisionTreeClassifier()
    dt.fit(train_x,train_y)
    y_pred = dt.predict(test_x)
    print("**** Decision Tree Classifier ****")
    print('Confusion Matrix')
    print(confusion_matrix(test_y,y_pred))
    print('Classification Report')
    print(classification_report(test_y,y_pred))

In [ ]:
def randomForest(train_x,test_x,train_y,test_y):
    rf = RandomForestClassifier()
    rf.fit(train_x,train_y)
    y_pred = rf.predict(test_x)
    print("**** Random Forest Classifier ****")
    print('Confusion Matrix')
    print(confusion_matrix(test_y,y_pred))
    print('Classification Report')
    print(classification_report(test_y,y_pred))

In [ ]:
def knn(train_x,test_x,train_y,test_y):
    knn = KNeighborsClassifier()
    knn.fit(train_x,train_y)
    y_pred = knn.predict(test_x)
    print("**** KNeighbour Classifier ****")
    print('Confusion Matrix')
    print(confusion_matrix(test_y,y_pred))
    print('Classification Report')
    print(classification_report(test_y,y_pred))

In [ ]:
def xgboost(train_x,test_x,train_y,test_y):
    xg = GradientBoostingClassifier()
    xg.fit(train_x,train_y)
    y_pred = xg.predict(test_x)
    print("**** Gradient Boosting Classifier ****")
    print('Confusion Matrix')
    print(confusion_matrix(test_y,y_pred))
    print('Classification Report')
    print(classification_report(test_y,y_pred))

In [ ]:
decisionTree(train_x,test_x,train_y,test_y)

In [ ]:
randomForest(train_x,test_x,train_y,test_y)

In [ ]:
knn(train_x,test_x,train_y,test_y)

In [ ]:
xgboost(train_x,test_x,train_y,test_y)

In [ ]:
rf = RandomForestClassifier()
rf.fit(train_x,train_y)
ypred = rf.predict(test_x)


In [ ]:
f1_score(ypred,test_y,average='weighted')

In [ ]:
cv = cross_val_score(rf,x,y,cv=5)

In [ ]:
np.mean(cv)

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient
import json

In [ ]:
###  Authenticate and set space

In [ ]:
wml_credentials = {
    "apikey" : "g9bGmoW9_jtSp9DcTiVZOCOyxcFLSvYG8DLIKyFpcSXt",
    "url" : "https://us-south.ml.cloud.ibm.com"
}

In [ ]:
wml_client = APIClient(wml_credentials)

In [ ]:
wml_client.spaces.list()

In [ ]:
SPACE_ID= "05f86e4f-db21-41b5-bfff-eb590af77a7b"

In [ ]:
wml_client.set.default_space(SPACE_ID)

In [ ]:
wml_client.software_specifications.list(100)

###  Save and deploy

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
MODEL_NAME = 'Model_building_SL'
DEPLOYMENT_NAME = 'new_smart_lender'
DEMO_MODEL = rf

In [ ]:
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [ ]:
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=train_x, 
    training_target=train_y
)

In [ ]:
model_details

In [ ]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

In [ ]:
#set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)